In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("C:\\Users\\GNANASEKARAN\\OneDrive\\Documents\\Python Project\\customer_transactions.csv")

In [3]:
df.head()

,Customer_ID,Order_ID,Order_Date,Quantity,Unit_Price,Revenue
0,1102,50000,2023-05-19,2,887,1774
1,1435,50001,2023-08-17,3,308,924
2,1860,50002,2023-10-01,3,366,1098
3,1270,50003,2023-02-19,1,755,755
4,1106,50004,2023-04-28,5,1096,5480


In [4]:
df.shape

(12000, 6)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_ID  12000 non-null  int64 
 1   Order_ID     12000 non-null  int64 
 2   Order_Date   12000 non-null  object
 3   Quantity     12000 non-null  int64 
 4   Unit_Price   12000 non-null  int64 
 5   Revenue      12000 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 562.6+ KB


In [6]:
df.describe()

,Customer_ID,Order_ID,Quantity,Unit_Price,Revenue
count,12000.000000,12000.00000,12000.000000,12000.000000,12000.000000
mean,1503.077750,55999.50000,3.031083,1047.327833,3175.354667
std,289.846514,3464.24595,1.421016,546.610521,2355.631330
min,1000.000000,50000.00000,1.000000,100.000000,100.000000
25%,1250.000000,52999.75000,2.000000,574.000000,1260.000000
50%,1506.000000,55999.50000,3.000000,1047.500000,2567.000000
75%,1757.000000,58999.25000,4.000000,1521.250000,4695.750000
max,1999.000000,61999.00000,5.000000,1999.000000,9985.000000


In [7]:
total_revenue = df["Revenue"].sum()
total_revenue

38104256

In [8]:
df["Customer_ID"].nunique()

1000

In [9]:
df["Revenue"].describe()

count    12000.000000
mean      3175.354667
std       2355.631330
min        100.000000
25%       1260.000000
50%       2567.000000
75%       4695.750000
max       9985.000000
Name: Revenue, dtype: float64

In [10]:
df["Order_Date"] = pd.to_datetime(df["Order_Date"])

In [11]:
customer_summary = (
    df.groupby("Customer_ID")
    .agg(
        total_revenue=("Revenue", "sum"),
        total_orders=("Order_ID", "nunique"),
        total_quantity=("Quantity", "sum")
    )
    .reset_index()
)

In [12]:
customer_summary.head()

,Customer_ID,total_revenue,total_orders,total_quantity
0,1000,63911,18,59
1,1001,38972,12,32
2,1002,43150,10,37
3,1003,23507,8,24
4,1004,46418,10,41


In [13]:
customer_summary.shape

(1000, 4)

In [14]:
customer_summary_sorted = customer_summary.sort_values(
    by="total_revenue",
    ascending=False
)

In [15]:
customer_summary_sorted.head()

,Customer_ID,total_revenue,total_orders,total_quantity
89,1089,88098,18,67
553,1553,87733,18,64
920,1920,85988,24,67
987,1987,84758,20,62
663,1663,84691,19,60


In [16]:
customer_summary_sorted["total_revenue"].sum()

38104256

In [17]:
total_revenue = customer_summary_sorted["total_revenue"].sum()
total_revenue

38104256

In [18]:
customer_summary_sorted["cumulative_revenue"] = (
    customer_summary_sorted["total_revenue"].cumsum()
)

In [19]:
customer_summary_sorted["cumulative_revenue_pct"] = (
    customer_summary_sorted["cumulative_revenue"] / total_revenue
)

In [20]:
customer_summary_sorted.head()

,Customer_ID,total_revenue,total_orders,total_quantity,cumulative_revenue,cumulative_revenue_pct
89,1089,88098,18,67,88098,0.002312
553,1553,87733,18,64,175831,0.004614
920,1920,85988,24,67,261819,0.006871
987,1987,84758,20,62,346577,0.009095
663,1663,84691,19,60,431268,0.011318


In [21]:
total_customers = customer_summary_sorted.shape[0]
top_20_cutoff = int(total_customers * 0.20)

top_20_cutoff

200

In [22]:
top_20_revenue = customer_summary_sorted.iloc[:top_20_cutoff]["total_revenue"].sum()
top_20_revenue_pct = top_20_revenue / total_revenue

top_20_revenue_pct

0.3096651985541983

In [23]:
customer_summary_sorted["customer_segment"] = "Bottom 80%"
customer_summary_sorted.loc[
    customer_summary_sorted.index[:top_20_cutoff],
    "customer_segment"
] = "Top 20%"

In [24]:
customer_summary_sorted["customer_segment"].value_counts()

Bottom 80%    800
Top 20%       200
Name: customer_segment, dtype: int64

In [25]:
final_customer_summary = customer_summary_sorted[[
    "Customer_ID",
    "total_revenue",
    "total_orders",
    "total_quantity",
    "customer_segment"
]]

In [26]:
final_customer_summary.head()

,Customer_ID,total_revenue,total_orders,total_quantity,customer_segment
89,1089,88098,18,67,Top 20%
553,1553,87733,18,64,Top 20%
920,1920,85988,24,67,Top 20%
987,1987,84758,20,62,Top 20%
663,1663,84691,19,60,Top 20%


In [27]:
final_customer_summary.to_excel(
    "C:\\Users\\GNANASEKARAN\\OneDrive\\Documents\\Python Project\\customer_revenue_summary.xlsx",
    index=False
)